# Auto-provision a new user

- Create a home directory
- Create NFS export
- Create SMB share
- Create quota
- Set up daily snapshots

### Prerequisites

- Install the qumulo api via `pip install qumulo_api`, or download it from your Qumulo cluster on the API & Tools page
- set up all the variables in the cell below

In [ ]:
cluster = 'XXXXX'    # Qumulo cluster hostname or IP where you're setting up users
api_user = 'XXXXX'     # Qumulo api user name
api_password = 'XXXXX' # Qumulo api password
base_dir = 'XXXXX' # the parent path where the users will be created.

user_name = 'XXXXX' # the new "user" to set up.

In [ ]:
import os
import sys
import traceback

from qumulo.rest_client import RestClient
from qumulo.rest.nfs import NFSRestriction

In [ ]:
full_path = '/'+ base_dir + '/' + user_name

In [ ]:
rc = RestClient(cluster, 8000)
rc.login(api_user, api_password)

In [ ]:
def create_dir(rc, name, dir_path='/'):
    try:
        rc.fs.create_directory(name = name, dir_path = dir_path)
    except:
        exc_type, exc_value, exc_traceback = sys.exc_info()
        print("Exception: %s" % exc_value)

In [ ]:
# Create base user directory, if it doesn't already exist
create_dir(rc, name=base_dir, dir_path='/')

### Create directory

In [ ]:
dir_res = rc.fs.create_directory(name=user_name, dir_path='/'+ base_dir)
print("Directory '%s' created with id: %s" % (full_path, dir_res['file_number']))
dir_id = dir_res['file_number']

### Create 20GB Quota

In [ ]:
quota_res = rc.quota.create_quota(id_ = dir_id, limit_in_bytes = 20000000000)

### Create NFS export

In [ ]:
nfs_res = rc.nfs.nfs_add_share(export_path = '/' + user_name,
                        fs_path = full_path,
                        description = "%s home directory" % user_name,
                        restrictions = [NFSRestriction({
                                            'read_only': False, 
                                            'host_restrictions': [],
                                            'user_mapping': 'NFS_MAP_NONE', 
                                            'map_to_user_id': '0'})]
                    )
print("NFS export created: %s with id %s" % (full_path, nfs_res['id']))

### Create SMB share

In [ ]:
smb_res = rc.smb.smb_add_share(share_name = user_name, 
                           fs_path = full_path, 
                           description = "%s home directory" % user_name
                          )
print("SMB share created: %s with id %s" % (full_path, smb_res['id']))

### Set up snapshot policy

In [ ]:
snap_res = rc.snapshot.create_policy(name = "User %s" % user_name, 
                                schedule_info = {"creation_schedule":
                                                 {"frequency":"SCHEDULE_DAILY_OR_WEEKLY",
                                                  "hour":2,"minute":15,
                                                  "on_days":["MON","TUE","WED","THU","FRI","SAT","SUN"],
                                                  "timezone":"America/Los_Angeles"},
                                                 "expiration_time_to_live":"7days"
                                                },
                                directory_id = str(dir_id),
                                enabled = True)
print("Snapshot policy created with id %s" % snap_res['id'])

### Clean up everything

In [ ]:
rc.quota.delete_quota(id_ = quota_res['id'])
rc.snapshot.delete_policy(policy_id = snap_res['id'])
rc.smb.smb_delete_share(id_ = smb_res['id'])
rc.nfs.nfs_delete_share(id_ = nfs_res['id'])
if full_path != '/': # small sanity check since tree delete is rather powerful.
    rc.fs.delete_tree(path = full_path)
print("Everything is cleaned up!")